In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]

In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

Age  EstimatedSalary  Purchased  Gender_Male
0     19            19000          0            1
1     35            20000          0            1
2     26            43000          0            0
3     27            57000          0            0
4     19            76000          0            1
..   ...              ...        ...          ...
395   46            41000          1            0
396   51            23000          1            1
397   50            20000          1            0
398   36            33000          0            1
399   49            36000          1            0

[400 rows x 4 columns]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection  import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test) 

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid={'weights':['uniform', 'distance', None],'algorithm':['auto', 'ball_tree', 'kd_tree','brute'],'n_neighbors':[5,7,9],'metric':['minkowski', None],'p':[2]}
grid=GridSearchCV( KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)


Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
44 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_param

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski', None],
                         'n_neighbors': [5, 7, 9], 'p': [2],
                         'weights': ['uniform', 'distance', None]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}: 0.9171245421245421


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[73  6]
 [ 4 37]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.9424205001543686

In [18]:
table=pd.DataFrame.from_dict(re)
table

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.034089      0.029936         0.026001        0.004147   
1        0.004229      0.002191         0.055539        0.049546   
2        0.007290      0.005830         0.022653        0.006867   
3        0.004285      0.004557         0.023583        0.004670   
4        0.007496      0.006284         0.013033        0.002745   
..            ...           ...              ...             ...   
67       0.000704      0.001408         0.000000        0.000000   
68       0.002052      0.001962         0.000000        0.000000   
69       0.000785      0.000967         0.000000        0.000000   
70       0.000374      0.000462         0.000000        0.000000   
71       0.000000      0.000000         0.000000        0.000000   

   param_algorithm param_metric param_n_neighbors param_p param_weights  \
0             auto    minkowski                 5       2       uniform   
1             auto    minkowski                 5       2      distance   
2             auto    minkowski                 5       2          None   
3             auto    minkowski                 7       2       uniform   
4             auto    minkowski                 7       2      distance   
..             ...          ...               ...     ...           ...   
67           brute         None                 7       2      distance   
68           brute         None                 7       2          None   
69           brute         None                 9       2       uniform   
70           brute         None                 9       2      distance   
71           brute         None                 9       2          None   

                                               params  split0_test_score  \
0   {'algorithm': 'auto', 'metric': 'minkowski', '...           0.874254   
1   {'algorithm': 'auto', 'metric': 'minkowski', '...           0.855314   
2   {'algorithm': 'auto', 'metric': 'minkowski', '...           0.874254   
3   {'algorithm': 'auto', 'metric': 'minkowski', '...           0.874254   
4   {'algorithm': 'auto', 'metric': 'minkowski', '...           0.855314   
..                                                ...                ...   
67  {'algorithm': 'brute', 'metric': None, 'n_neig...                NaN   
68  {'algorithm': 'brute', 'metric': None, 'n_neig...                NaN   
69  {'algorithm': 'brute', 'metric': None, 'n_neig...                NaN   
70  {'algorithm': 'brute', 'metric': None, 'n_neig...                NaN   
71  {'algorithm': 'brute', 'metric': None, 'n_neig...                NaN   

    split1_test_score  split2_test_score  split3_test_score  \
0            0.874254           0.876643           0.929144   
1            0.855314           0.859435           0.929144   
2            0.874254           0.876643           0.929144   
3            0.874254           0.859435           0.947015   
4            0.874254           0.859435           0.929144   
..                ...                ...                ...   
67                NaN                NaN                NaN   
68                NaN                NaN                NaN   
69                NaN                NaN                NaN   
70                NaN                NaN                NaN   
71                NaN                NaN                NaN   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
0            0.946153         0.900089        0.031147                9  
1            0.946153         0.889072        0.040054               29  
2            0.946153         0.900089        0.031147                9  
3            0.964286         0.903849        0.042989                1  
4            0.946153         0.892860        0.037496               25  
..                ...              ...             ...              ...  
67                NaN              NaN             NaN               37  
68                NaN   